In [77]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key


'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'

### Initialize the ai model

In [78]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama3-8b-8192",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000016842859010>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001684285B290>, model_name='llama3-8b-8192', groq_api_key=SecretStr('**********'))

### Load the Data and Create Documents

In [79]:
import pandas as pd
from langchain.docstore.document import Document
df = pd.read_excel('data.xlsx')
documents = []

for _, row in df.iterrows():
    content = row.to_string()  # Convert the row to a string
    doc = Document(page_content=content)
    documents.append(doc)


In [80]:
documents

[Document(page_content='Incident Category                                                Cyber security\nincident severity levels                                                   High\nIncident Name                                           Phishing Email Campaign\nDescription                   *On : 2023-11-11\\nReceived a fraudulent email ...\nAction and  Recommendation    immediately report to IT security team, quaran...\nRoot-Cause                    * Exploited vulnerabilities: SQL injection, cr...\nType                                                                   Phishing\nImpact                                                          Confidentiality\nInvestigator                                                         Ahmed Omer'),
 Document(page_content='Incident Category                                                Cyber security\nincident severity levels                                                 Medium\nIncident Name                                             

### Initialize the Embedding Model

In [81]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\Hima\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Create the Vector Store

In [82]:
vector_store = FAISS.from_documents(documents, embeddings)
vector_store.save_local('faiss_index')

### Create a Prompt for the AI to Search the Vector Store

In [83]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

retriever=vector_store.as_retriever(
    search_type="similarity",
)

message = """
Answer this question using the provided context only and give the users all Actions and Recommendations needed to solve the problem.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|model


In [84]:
user_input = "I received a fraudulent email impersonating the CEO, requesting immediate transfer of funds and there are some employees telling me that unidentified malware infected their systems, encrypting critical data and demanding a ransom. what should i do?"

response=rag_chain.invoke(user_input)
print(response.content)

Based on the provided context, here are the actions and recommendations to handle the situation:

**Immediate Actions:**

1. **Isolate Infected Systems**: Immediately isolate the systems that have been infected with the ransomware malware to prevent further spread of the malware and potential data loss.
2. **Notify the CEO and Incident Response Team**: Inform the CEO and the incident response team about the fraudulent email and the ransomware attack.
3. **Conduct a Thorough Investigation**: Conduct a thorough investigation to determine the root cause of the incident, including reviewing the email headers, monitoring logs, and interviewing affected employees.

**Recommendations:**

1. **Implement Regular Backups**: Implement regular backups of critical data to ensure that data can be restored in case of a ransomware attack.
2. **Ensure Backups are Secure**: Ensure that backups are stored securely and are not accessible to unauthorized individuals.
3. **Implement Email Authentication Pro

In [85]:
retriever.batch([user_input])

[[Document(page_content='Incident Category                                                Cyber security\nincident severity levels                                                 Medium\nIncident Name                                                 Ransomware Attack\nDescription                   * On 2023-11-15\\nUnidentified malware infected...\nAction and  Recommendation    Isolate infected systems, conduct a thorough i...\nRoot-Cause                    * Phishing email bypassed email security filte...\nType                                                                    Malware\nImpact                                                          Confidentiality\nInvestigator                                                    mahmoud Mohamed'),
  Document(page_content="Incident Category                                                Cyber security\nincident severity levels                                               Critical\nIncident Name                 Ransomware Attack (Data En